In [62]:
import sqlite3

In [71]:
# client = OpenAI(api_key='sk-proj-tmR5ExbMdnZeFVs7OgH7T3BlbkFJLwrPmPTMiznz6IZJQBuA')

principles = {
    1: 'If you prefer more concise answers, no need to be polite with LLM so there is no need to add phrases like “please", "if you don"t mind", "thank you", "I would like to", etc., and get straight to the point.',
    2: "Integrate the intended audience in the prompt, e.g., the audience is an expert in the field.",
    3: "Break down complex tasks into a sequence of simpler prompts in an interactive conversation.",
    4: "Employ affirmative directives such as 'do,' while steering clear of negative language like 'don't'.",
    5: '''When you need clarity or a deeper understanding of a topic, idea, or any piece of information, utilize the following prompts:
        - Explain [insert specific topic] in simple terms.
        - Explain to me like I'm 11 years old.
        - Explain to me as if I'm a beginner in [field].
        - Explain to me as if I'm an expert in [field].
        - "Write the [essay/text/paragraph] using simple English like you're explaining something to a 5-year-old".''',
    6: "Include phrases like 'I'm going to tip $ for a better solution/answer!",
    7: "Implement example-driven prompting (Use few-shot prompting).",
    8: "When formatting your prompt, start with '###Instruction###', followed by either '###Example###' or '###Question###' if relevant. Subsequently, present your content. Use one or more line breaks to separate instructions, examples, questions, context, and input data.",
    9: "Incorporate the following phrases: 'Your task is' and 'You MUST'.",
    10:"Incorporate the following phrases: 'You will be penalized'.",
    11:"Include the phrase 'Answer a question given in a natural, human-like manner.' at the end of each question.",
    12:"Use the phrase 'Answer a question given in a natural, human-like manner' in your prompts.",
    13:"Include the following phrase 'Ensure that your answer is unbiased and does not rely on stereotypes' in each question.",    
    14:'Allow the model to elicit precise details and requirements from you by asking you questions until it has enough information to provide the needed output (for example, "From now on, I would like you to ask me questions to...").', 
    15:'To inquire about a specific topic or idea or any information and you want to test your understanding, you can use the following phrase: "Teach me the [Any theorem / topic / rule name] and include a test at the end, but don"t give me the answers and then tell me if I got the answer right when I respond".',
    16:"Assign a role to the Large Language Models (LLMs).",
    17: "Use Delimiters, like quotation marks to give emphasis for specific words.",
    18: "Repeat a specific word or phrase multiple times within a prompt.",
    19: "Combine Chain-of-thought (Cot) with few-Shot prompts.", 
    20: "Use output primers, which involve concluding your prompt with the beginning of the desired output. Utilize output primers by ending your prompt with the start of the anticipated response.",
    21: "To write an [essay / text paragraph / article] or any type of text that should be detailed: 'Write a detailed [essay / text / paragraph] for me on [topic] in detail by adding all the information necessary'.",
    22: 'To correct / change specific text without changing its style: "Try to revise every paragraph sent by users. You should only improve the user’s grammar and vocabulary and make sure it sounds natural. You should maintain the original writing style, ensuring that a formal paragraph remains formal".',
    23: 'When you have a complex coding prompt that may be in different files: "From now and on whenever you generate code that spans more than one file, generate a [programming language] script that can be run to automatically create the specified files or make changes to existing files to insert the generated code. [your question]."',
    24: "When you want to initiate or continue a text using specific words, phrases, or sentences, utilize the following prompt:- I'm providing you with the beginning [song lyrics / story / paragraph / essay...]: [Insert lyrics / words / sentence]. Finish it based on the words provided. Keep the flow consistent.",
    25: "Clearly state the requirements that the model must follow in order to produce content, in the form of keywords, regulations, hints, or instructions.",
    26: 'To write any text, such as an essay or paragraph, that is intended to be similar to a provided sample, include the following instructions:- "Use the same language based on the provided paragraph[ / title / text / essay / answer]".'
}

In [72]:
def create_chat_prompt(res1, res2, pid=1):
    principle = principles[pid]
    # sys_msg = """Choose the better-written option. The first character of your response must be 1 for response 1 or 2 for response 2. After the first character, give a brief justification for your choice."""
    sys_msg = """Your task is to compare two responses and determine which one is better based on the criteria listed below. Ensure that the evaluation is aligned with the provided principle. Prompt following the principle gives option-1 response and option-2 response is obtained without using the principle on the promp. Choose the response that is correct. If both responses are correct, assess based on the provision of more information, clarity, adherence to the prompt instructions, and how well the response reflects the user's query and principle. If both responses are the same, indicate that Option 1 is equal to Option 2. The first character of your response must be 1 for response 1 or 2 for response 2 or 0 if both are the same. After the first character, give a brief justification for your choice."""
    # user_prompt = f"""Option 1: {res1}
                    #  Option 2: {res2} """
    user_prompt = f"""###Instruction###
        {sys_msg}
        
        ###Principle###
        {principle}
        
        ###Criteria###
        1. Correctness: Is the response correct?
        2. Provision of Information: Does the response provide more comprehensive information?
        3. Adherence to Prompt Instructions: Does the response follow the prompt instructions closely?
        4. Reflection of User's Query: How well does the response reflect and address the user's query?
        
        ###Responses###
        Option 1: {res1}
        Option 2: {res2}
        
        ###Question###
        Based on the criteria listed above, which response is better and why? Please provide a brief justification after indicating your choice with '1' for Option 1 or '2' for Option 2 or '0' if both are the same.
        """
    
    return [
        {"role": "system", "content": sys_msg}, 
        {"role": "user", "content": user_prompt}
    ]

In [73]:
db_path = './DBs/principles.db'
def get_all_rows():
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    rows = []
    for i in range(1, 27): 
        if i in [3, 14, 19, 22, 23]:
            continue
        table_name = f'principle{i}'
        c.execute(f'SELECT * FROM {table_name}')
        rows.extend(c.fetchall())
    conn.close()
    return rows

In [74]:
import os
import json

model = 'gpt-4o'

filename = f"{model}_requests.jsonl"
if os.path.exists(filename):
    os.remove(filename)
    os.remove(filename.replace(".jsonl", "_results.jsonl"))
rows = get_all_rows()
jobs = [create_chat_prompt(row[4], row[5]) for row in rows]
with open(filename, "w") as f:
    for job in jobs:
        body = {"model": model, "messages": job}
        json_string = json.dumps(body)
        f.write(json_string + "\n")

In [75]:
!python api_request_parallel_processor.py --requests_filepath 'gpt-4o_requests.jsonl'

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [76]:
import json

# Load the JSONL files
file1_path = "gpt-4o_requests.jsonl"
file2_path = "gpt-4o_requests_results.jsonl"


with open(file1_path, 'r') as file:
    file1_entries = [json.loads(line) for line in file]

with open(file2_path, 'r') as file:
    file2_entries = [json.loads(line) for line in file]

# k = 1/0
if len(file1_entries) != len(file2_entries):
    raise ValueError('The length of the request and response files are not same!')
# print(file2_entries[0][0]['messages'][1]['content'])
# k = 1/0
ordered_file2_entries = []
for i, entry1 in enumerate(file1_entries):
    for entry2 in file2_entries:
        if entry2[0]['messages'][1]['content'] == entry1['messages'][1]['content']:
            ordered_file2_entries.append(entry2)
            break

# Save the ordered entries to a new JSONL file
ordered_file2_path = f"{model}_ordered_response.jsonl"
with open(ordered_file2_path, 'w') as file:
    for entry in ordered_file2_entries:
        file.write(json.dumps(entry) + '\n')

print(f"Ordered file saved to {ordered_file2_path}")


Ordered file saved to gpt-4o_ordered_response.jsonl


### Store preference in database

In [77]:
def extract_res(text):
    idx1 = text.find('Option 1: ')
    idx2 = text.find('Option 2: ')
    return text[idx1+10:idx2]

def update_preferences(response1, machine_preference, justification=None):
    flag = 0
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    for i in range(1, 27): 
        table_name = f'principle{i}'
        c.execute(f'SELECT  qid, response_with_principle FROM {table_name}')
        all_response1 = c.fetchall()
        
        for qid, res in all_response1:
            # print(res)
            # print(response1)
            # check_where(res, response1)
            if res.strip(' \n') == response1.strip(' \n'):
                # print('Great')
                c.execute(f"UPDATE {table_name} SET machine_preference=? WHERE qid=?", (machine_preference,  qid))
                conn.commit()
                c.execute(f"UPDATE {table_name} SET machine_pref_justification=? WHERE qid=?", (justification, qid))
                conn.commit()
                flag = 1
                break
            
        if flag == 1:
            break
        
        # break
    conn.close()
    return flag


In [79]:
file_path = f"{model}_ordered_response.jsonl"


with open(file_path, 'r') as file:
    file_entries = [json.loads(line) for line in file]

for i, entry in enumerate(file_entries):
    text = entry[0]['messages'][1]['content']
    preference = entry[1]['choices'][0]['message']['content'][0]
    justification = entry[1]['choices'][0]['message']['content'][2:]
    response1 = extract_res(text)
    # print(response1)
    flag = update_preferences(response1, preference, justification)
    if flag == 0:
        print(f'Entry: {i} is not updated in database. Do it manually')
    # break
    
